In [1]:
import pandas as pd
import numpy as np # Required for np.inf

# --- Configuration ---
# FILE_PATH is implied as 'car_prices.csv' when uploaded to this environment
CLEANED_FILE_PATH = 'cleaned_car_prices_final_dateonly.csv' # Consistent output filename

print(f"Loading data from: {'car_prices.csv'}")

# 1. Load the dataset
try:
    df = pd.read_csv('car_prices.csv')
    print("Data loaded successfully!")
    print(f"Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: {'car_prices.csv'} not found. Please ensure the file is uploaded.")
    exit()

# --- Data Cleaning and Preprocessing ---

# Drop unnecessary columns that were consistently identified as not present or not needed
columns_to_drop_actual = []
df = df.drop(columns=columns_to_drop_actual, errors='ignore')
print(f"Shape after dropping initial unneeded columns: {df.shape}")

# Drop rows where 'sellingprice' or 'saledate' is missing (critical for analysis)
df = df.dropna(subset=['sellingprice', 'saledate']).copy() # Added .copy() to avoid SettingWithCopyWarning
print(f"Shape after dropping NaNs in 'sellingprice' or 'saledate': {df.shape}")

# Filter out rows where sellingprice is 0 or less (assuming valid car prices are positive)
df = df[df['sellingprice'] > 0].copy() # Added .copy()
print(f"Shape after filtering out sellingprice <= 0: {df.shape}")

# --- CRITICAL FIXES: Rename columns and ensure numeric types ---
# Rename 'sellingprice' to 'price' and 'make' to 'manufacturer'
df = df.rename(columns={'sellingprice': 'price', 'make': 'manufacturer'})
print(f"Shape after renaming 'sellingprice' to 'price' and 'make' to 'manufacturer': {df.shape}")

# Ensure 'price' is numeric and handle any non-numeric values that might appear
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.dropna(subset=['price'], inplace=True) # This is fine as it's on the Series directly
print(f"Shape after ensuring 'price' is numeric and handling NaNs: {df.shape}")

# Handle missing 'condition', 'odometer', 'mmr' by median imputation
for col in ['condition', 'odometer', 'mmr']:
    if col in df.columns and df[col].isnull().any():
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val) # Direct assignment handles inplace warning
print(f"Shape after median imputation for 'condition', 'odometer', 'mmr': {df.shape}")

# Ensure 'year' is integer and handle potential NaNs
if 'year' in df.columns:
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df.dropna(subset=['year'], inplace=True)
    df['year'] = df['year'].astype(int)
    print("Ensured 'year' column is integer and handled NaNs.")

# --- CRITICAL SALEDATE PARSING ---
# Added utc=True to handle mixed time zones and ensure a consistent datetime dtype.
try:
    df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce', utc=True) # *** FIX IS HERE ***
    print(f"Shape after converting 'saledate' to datetime objects (before dropping NaTs): {df.shape}")

    # Drop rows where saledate conversion failed (resulted in NaT)
    df.dropna(subset=['saledate'], inplace=True)
    print(f"Shape after dropping failed 'saledate' conversions: {df.shape}")

    # Now, convert to a string format (YYYY-MM-DD) that Power BI will easily recognize as a Date
    # .dt accessor will now work as column should be datetime64[ns, UTC]
    df['saledate'] = df['saledate'].dt.strftime('%Y-%m-%d')
    print(f"Shape after formatting 'saledate' as YYYY-MM-DD string: {df.shape}")

except AttributeError as e:
    print(f"Error converting 'saledate' or accessing .dt accessor: {e}")
    print("This usually means 'saledate' did not convert to datetime objects.")
    print("Please provide 5-10 raw examples of 'saledate' values from your CSV file.")
    exit()

# For categorical columns, fill NaNs with 'Unknown'
cols_to_fill_unknown = ['region', 'model', 'condition', 'cylinders', 'fuel',
                        'title_status', 'transmission', 'drive', 'type', 'paint_color', 'state']
for col in cols_to_fill_unknown:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown').astype(str) # Chained fillna and astype for direct assignment
print(f"Shape after filling categorical NaNs with 'Unknown': {df.shape}")


# --- Feature Engineering ---

# Calculate 'Car_Age'
CURRENT_YEAR = pd.Timestamp.now().year
if 'year' in df.columns:
    df['Car_Age'] = CURRENT_YEAR - df['year']
    df = df[df['Car_Age'] >= 0].copy() # Added .copy()
    print(f"Shape after creating 'Car_Age' and filtering for non-negative age: {df.shape}")

# Create Price Bins
if 'price' in df.columns:
    bins_price = [0, 10000, 20000, 30000, 50000, np.inf]
    labels_price = ['0-10k', '10k-20k', '20k-30k', '30k-50k', '50k+']
    df['Price_Range'] = pd.cut(df['price'], bins=bins_price, labels=labels_price, right=False).astype('category')
    print(f"Shape after creating 'Price_Range' bins: {df.shape}")

# Create Mileage Bins
if 'odometer' in df.columns:
    bins_mileage = [0, 50000, 100000, 150000, np.inf]
    labels_mileage = ['0-50k', '50k-100k', '100k-150k', '150k+']
    df['Mileage_Range'] = pd.cut(df['odometer'], bins=bins_mileage, labels=labels_mileage, right=False).astype('category')
    print(f"Shape after creating 'Mileage_Range' bins: {df.shape}")

# Select and reorder columns for clarity and consistency
final_columns = [
    'year', 'manufacturer', 'model', 'trim', 'body', 'transmission', 'vin',
    'state', 'condition', 'odometer', 'color', 'interior', 'seller',
    'mmr', 'price', 'saledate', 'Car_Age', 'Price_Range', 'Mileage_Range'
]

final_columns_present = [col for col in final_columns if col in df.columns]
df_cleaned_final = df[final_columns_present].copy()
print(f"Final DataFrame shape before saving: {df_cleaned_final.shape}")

# 5. Export the Cleaned Data for Power BI
df_cleaned_final.to_csv(CLEANED_FILE_PATH, index=False)
print(f"\nCleaned data successfully exported to '{CLEANED_FILE_PATH}'")

Loading data from: car_prices.csv
Data loaded successfully!
Initial shape: (558837, 16)
Shape after dropping initial unneeded columns: (558837, 16)
Shape after dropping NaNs in 'sellingprice' or 'saledate': (558825, 16)
Shape after filtering out sellingprice <= 0: (558825, 16)
Shape after renaming 'sellingprice' to 'price' and 'make' to 'manufacturer': (558825, 16)
Shape after ensuring 'price' is numeric and handling NaNs: (558825, 16)
Shape after median imputation for 'condition', 'odometer', 'mmr': (558825, 16)
Ensured 'year' column is integer and handled NaNs.


C:\Users\kavya\AppData\Local\Temp\ipykernel_52644\4234414119.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce', utc=True) # *** FIX IS HERE ***


Shape after converting 'saledate' to datetime objects (before dropping NaTs): (558825, 16)
Shape after dropping failed 'saledate' conversions: (558799, 16)
Shape after formatting 'saledate' as YYYY-MM-DD string: (558799, 16)
Shape after filling categorical NaNs with 'Unknown': (558799, 16)
Shape after creating 'Car_Age' and filtering for non-negative age: (558799, 17)
Shape after creating 'Price_Range' bins: (558799, 18)
Shape after creating 'Mileage_Range' bins: (558799, 19)
Final DataFrame shape before saving: (558799, 19)

Cleaned data successfully exported to 'cleaned_car_prices_final_dateonly.csv'
